In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet, HuberRegressor
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.model_selection import LeaveOneGroupOut
import random
from scipy.stats import loguniform
from scipy.stats import uniform
from sklearn.preprocessing import StandardScaler
from scipy.stats import spearmanr, pearsonr
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import LinearRegression, SGDRegressor,BayesianRidge
from torch.utils.data import Dataset, TensorDataset
from torch.utils.data import DataLoader
# import pybnesian as pbn

from shapely import wkt
from datetime import datetime

from sklearn.neighbors import KernelDensity
from IPython.display import clear_output
import matplotlib.pyplot as plt
import scipy
import numpy as np
from scipy.stats import gamma as scipy_gamma
from scipy.special import gamma as gamma_function

/home/medourahou1/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
# Constants
legs_input_path = '../Cities/Padam_terretory_01/Ressources/cleaned_padam__drt_trips_terretory_01.csv' # MATSim output legs csv file
 
# popArea_input_path = 'shp/CentresOfInterest_great_paris.shp' # the population of paris 


# EPSG code !!ONLY NUMBER!! EPSG:3035 --> 3035
crs_import = 2154 #2154 # crs of the raw imported data
crs_working = 2154 #2154 # crs used for the processing (can be same as crs_import)

# Toggle for preprocessing steps
# GTFS preprocessing to get hexagonal grid
#      --> needed once, can be skipped if GTFScells.csv file exists
toggle_GTFSprep = False
GTFScells_path = '../Cities/Padam_terretory_01/Ressources/cells_of_padam_terretory_01.csv'


# Interpolation will only be done in this time window
start_time = '06:00:00'
end_time = '22:00:00'
# Length of each analysis interval (the estimation of travel time and wait time will be done for each deltaT interval)
deltaT = 1 # in hours (h), default 1h

In [3]:
trips_df= pd.read_csv(legs_input_path, sep=",")
dataset = trips_df[['departure_latitude_x', 'departure_longitude_x', 'arrival_latitude_x',
       'arrival_longitude_x', 'departure_time_hour',
       'departure_time_minute', 'departure_time_seconds',
       'departure_time_day_of_week', 'departure_time_day_of_month',
       'departure_time_month', 'departure_time_hour_sin',
       'departure_time_hour_cos', 'departure_time_day_of_week_sin',
       'departure_time_day_of_week_cos', 'departure_time_month_sin',
       'departure_time_month_cos', 'route_distance','travel_time']]

dataset.rename(columns = {   "departure_latitude_x" : 'departure_latitude',
                            "departure_longitude_x" : "departure_longitude",
                            "arrival_latitude_x" : 'arrival_latitude',
                            "arrival_longitude_x" : 'arrival_longitude'
                        
                          },inplace=True)

dataset2 = dataset[dataset["travel_time"]<=2500]
dataset3 =dataset2[dataset2["travel_time"]>=60]

/tmp/ipykernel_1596638/1865277605.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.rename(columns = {   "departure_latitude_x" : 'departure_latitude',


In [4]:
dataset3 = dataset3[['departure_latitude', 'departure_longitude', 'arrival_latitude',
       'arrival_longitude', 'departure_time_hour','departure_time_day_of_week',
       'departure_time_day_of_month', 'departure_time_month','route_distance', 'travel_time']]

In [8]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import scipy.stats

class HistoricalAveragePredictor:
    def __init__(self, segment_by_time=True):
        self.segment_by_time = segment_by_time
        self.route_means = {}
        self.route_time_means = {}
        
    def create_route_key(self, row):
        """Create a unique key for each route based on departure and arrival coordinates"""
        return f"{row['departure_latitude']:.4f}_{row['departure_longitude']:.4f}_" + \
               f"{row['arrival_latitude']:.4f}_{row['arrival_longitude']:.4f}"
    
    def create_time_key(self, row):
        """Create a time segment key based on hour and day of week"""
        return f"{row['departure_time_day_of_week']}_{row['departure_time_hour']}"
    
    def fit(self, train_df):
        """Calculate historical averages for each route and time segment"""
        # Calculate route-level means
        for _, row in train_df.iterrows():
            route_key = self.create_route_key(row)
            if self.segment_by_time:
                time_key = self.create_time_key(row)
                full_key = f"{route_key}_{time_key}"
                
                if full_key not in self.route_time_means:
                    self.route_time_means[full_key] = []
                self.route_time_means[full_key].append(row['travel_time'])
            
            if route_key not in self.route_means:
                self.route_means[route_key] = []
            self.route_means[route_key].append(row['travel_time'])
        
        # Convert lists to means
        self.route_means = {k: np.mean(v) for k, v in self.route_means.items()}
        if self.segment_by_time:
            self.route_time_means = {k: np.mean(v) for k, v in self.route_time_means.items()}
        
        # Calculate global mean for unknown routes
        self.global_mean = train_df['travel_time'].mean()
        
    def predict(self, test_df):
        """Predict travel times using historical averages"""
        predictions = []
        
        for _, row in test_df.iterrows():
            route_key = self.create_route_key(row)
            
            if self.segment_by_time:
                time_key = self.create_time_key(row)
                full_key = f"{route_key}_{time_key}"
                
                if full_key in self.route_time_means:
                    predictions.append(self.route_time_means[full_key])
                elif route_key in self.route_means:
                    predictions.append(self.route_means[route_key])
                else:
                    predictions.append(self.global_mean)
            else:
                predictions.append(self.route_means.get(route_key, self.global_mean))
        
        return np.array(predictions)
    
    def evaluate(self, test_df):
        """Evaluate model performance"""
        predictions = self.predict(test_df)
        actual = test_df['travel_time'].values
        
        mae = mean_absolute_error(actual, predictions)
        rmse = np.sqrt(mean_squared_error(actual, predictions))
        mape = np.mean(np.abs((actual - predictions) / actual)) * 100
        r2 = r2_score(actual, predictions)
        
        print(f"Historical Average Model Performance:")
        print(f"MAE: {mae:.2f} seconds")
        print(f"RMSE: {rmse:.2f} seconds")
        print(f"MAPE: {mape:.2f}%")
        print(f"R² Score: {r2:.4f}")
        
        return {
            'mae': mae,
            'rmse': rmse,
            'mape': mape,
            'r2': r2,
            'predictions': predictions
        }

class RandomForestTravelTimePredictor:
    def __init__(self, n_estimators=100, max_depth=None, random_state=42):
        self.model = RandomForestRegressor(
            n_estimators=n_estimators,
            max_depth=max_depth,
            random_state=random_state,
            n_jobs=-1  # Use all available cores
        )
        self.base_features = [
            'departure_latitude', 'departure_longitude',
            'arrival_latitude', 'arrival_longitude',
            'departure_time_hour', 'departure_time_day_of_week',
            'departure_time_day_of_month', 'departure_time_month',
            'route_distance'
        ]
    
    def add_cyclic_features(self, df):
        """Add cyclic time features to the dataframe"""
        df = df.copy()
        
        # Hour features (24-hour cycle)
        df['departure_time_hour_sin'] = np.sin(2 * np.pi * df['departure_time_hour'] / 24)
        df['departure_time_hour_cos'] = np.cos(2 * np.pi * df['departure_time_hour'] / 24)
        
        # Day of week features (7-day cycle)
        df['departure_time_day_of_week_sin'] = np.sin(2 * np.pi * df['departure_time_day_of_week'] / 7)
        df['departure_time_day_of_week_cos'] = np.cos(2 * np.pi * df['departure_time_day_of_week'] / 7)
        
        # Month features (12-month cycle)
        df['departure_time_month_sin'] = np.sin(2 * np.pi * df['departure_time_month'] / 12)
        df['departure_time_month_cos'] = np.cos(2 * np.pi * df['departure_time_month'] / 12)
        
        return df
    
    def prepare_data(self, df):
        """Prepare features for the model"""
        # Add cyclic features
        df = self.add_cyclic_features(df)
        
        # Combine base features with cyclic features
        all_features = self.base_features + [
            'departure_time_hour_sin', 'departure_time_hour_cos',
            'departure_time_day_of_week_sin', 'departure_time_day_of_week_cos',
            'departure_time_month_sin', 'departure_time_month_cos'
        ]
        
        # Extract features and handle missing values
        features = df[all_features].values
        features = np.nan_to_num(features, nan=0.0, posinf=0.0, neginf=0.0)
        return features
    
    def fit(self, train_df):
        """Train the Random Forest model"""
        X_train = self.prepare_data(train_df)
        y_train = train_df['travel_time'].values
        
        # Train the model
        self.model.fit(X_train, y_train)
        
        # Calculate feature importance
        all_features = self.base_features + [
            'departure_time_hour_sin', 'departure_time_hour_cos',
            'departure_time_day_of_week_sin', 'departure_time_day_of_week_cos',
            'departure_time_month_sin', 'departure_time_month_cos'
        ]
        
        self.feature_importance = pd.DataFrame({
            'feature': all_features,
            'importance': self.model.feature_importances_
        }).sort_values('importance', ascending=False)
    
    def predict(self, test_df):
        """Make predictions using the Random Forest model"""
        X_test = self.prepare_data(test_df)
        predictions = self.model.predict(X_test)
        return predictions
    
    def evaluate(self, test_df):
        """Evaluate model performance"""
        predictions = self.predict(test_df)
        actual = test_df['travel_time'].values
        
        mae = mean_absolute_error(actual, predictions)
        rmse = np.sqrt(mean_squared_error(actual, predictions))
        mape = np.mean(np.abs((actual - predictions) / actual)) * 100
        r2 = r2_score(actual, predictions)
        
        # Calculate prediction intervals using quantile regression forest
        tree_predictions = np.array([tree.predict(self.prepare_data(test_df)) 
                                   for tree in self.model.estimators_])
        lower_bounds = np.percentile(tree_predictions, 2.5, axis=0)
        upper_bounds = np.percentile(tree_predictions, 97.5, axis=0)
        
        print(f"Random Forest Model Performance:")
        print(f"MAE: {mae:.2f} seconds")
        print(f"RMSE: {rmse:.2f} seconds")
        print(f"MAPE: {mape:.2f}%")
        print(f"R² Score: {r2:.4f}")
        
        print("\nTop 5 Most Important Features:")
        print(self.feature_importance.head())
        
        return {
            'mae': mae,
            'rmse': rmse,
            'mape': mape,
            'r2': r2,
            'predictions': predictions,
            'lower_bounds': lower_bounds,
            'upper_bounds': upper_bounds,
            'feature_importance': self.feature_importance
        }

# Example usage:
"""
# Initialize models
hist_avg_model = HistoricalAveragePredictor(segment_by_time=True)
rf_model = RandomForestTravelTimePredictor(n_estimators=100)

# Train models
hist_avg_model.fit(train_df)
rf_model.fit(train_df)

# Evaluate models
hist_results = hist_avg_model.evaluate(test_df)
rf_results = rf_model.evaluate(test_df)
"""

'\n# Initialize models\nhist_avg_model = HistoricalAveragePredictor(segment_by_time=True)\nrf_model = RandomForestTravelTimePredictor(n_estimators=100)\n\n# Train models\nhist_avg_model.fit(train_df)\nrf_model.fit(train_df)\n\n# Evaluate models\nhist_results = hist_avg_model.evaluate(test_df)\nrf_results = rf_model.evaluate(test_df)\n'

In [9]:
train_data, test_data = train_test_split(dataset3, test_size=0.2, random_state=42)


# Initialize models
hist_avg_model = HistoricalAveragePredictor(segment_by_time=True)
rf_model = RandomForestTravelTimePredictor(n_estimators=100)

# Train models
hist_avg_model.fit(train_data)
rf_model.fit(train_data)

# Evaluate models
hist_results = hist_avg_model.evaluate(test_data)
rf_results = rf_model.evaluate(test_data)


Historical Average Model Performance:
MAE: 191.53 seconds
RMSE: 279.82 seconds
MAPE: 32.55%
R² Score: 0.5332
Random Forest Model Performance:
MAE: 188.87 seconds
RMSE: 270.48 seconds
MAPE: 32.85%
R² Score: 0.5638

Top 5 Most Important Features:
                        feature  importance
8                route_distance    0.577868
6   departure_time_day_of_month    0.116595
7          departure_time_month    0.030580
13     departure_time_month_sin    0.028074
14     departure_time_month_cos    0.027435
